In [1]:
!pip install transformers datasets evaluate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" a

In [2]:
from datasets import load_dataset

# Load the "sms_spam" dataset
dataset = load_dataset("sms_spam")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})


In [4]:
# Look at the first few examples
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])

{'sms': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'label': 0}
{'sms': 'Ok lar... Joking wif u oni...\n', 'label': 0}
{'sms': "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n", 'label': 1}


In [5]:
from transformers import AutoTokenizer

# Load a pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's tokenize one example for fun
example_text = dataset['train'][0]['sms']
tokens = tokenizer(example_text)

print(tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 2175, 2127, 18414, 17583, 2391, 1010, 4689, 1012, 1012, 2800, 2069, 1999, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 1012, 1012, 1012, 25022, 2638, 2045, 2288, 26297, 28194, 1012, 1012, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
# Define a function to tokenize a batch of examples
def tokenize_batch(batch):
    return tokenizer(batch['sms'], padding="max_length", truncation=True)

# Apply the tokenizer to the entire dataset
tokenized_dataset = dataset.map(tokenize_batch, batched=True)

# Let's see what a tokenized example looks like now
print(tokenized_dataset['train'][0])

Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

{'sms': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'label': 0, 'input_ids': [101, 2175, 2127, 18414, 17583, 2391, 1010, 4689, 1012, 1012, 2800, 2069, 1999, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 1012, 1012, 1012, 25022, 2638, 2045, 2288, 26297, 28194, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
from transformers import AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # We have 2 labels: spam (1) and ham (0)
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments

# Set up training parameters
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save model checkpoints
    learning_rate=2e-5,               # Very small learning rate (safe for fine-tuning)
    per_device_train_batch_size=16,   # Batch size per GPU/CPU
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=3,               # Number of passes through the dataset
    weight_decay=0.01,                # Regularization to prevent overfitting
    logging_dir="./logs",             # Directory to save logs
)

In [9]:
from transformers import Trainer, TrainingArguments

# We also need a simple function to compute metrics like accuracy
import numpy as np
from datasets import load_metric

# Define a metric function (computes accuracy)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

# Create the Trainer
trainer = Trainer(
    model=model,                       # Our DistilBERT model
    args=training_args,                # Our training parameters
    train_dataset=tokenized_dataset['train'],  # Our tokenized dataset
    compute_metrics=compute_metrics,   # Function to compute evaluation metrics
)

ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)

In [10]:
!pip install evaluate

In [11]:
from transformers import Trainer
import numpy as np
from evaluate import load

# Load the accuracy metric
metric = load("accuracy")

# Define a metric function (computes accuracy)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Create the Trainer (note the report_to="none"!)
trainer = Trainer(
    model=model,                        # Our DistilBERT model
    args=training_args,                 # Our training parameters
    train_dataset=tokenized_dataset['train'],  # Our tokenized dataset
    compute_metrics=compute_metrics,    # Function to compute evaluation metrics
)

In [12]:
!pip install wandb

In [13]:
import wandb
wandb.init(mode="disabled")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,0.056800
1000,0.011500


TrainOutput(global_step=1047, training_loss=0.03352261301211892, metrics={'train_runtime': 770.0093, 'train_samples_per_second': 21.717, 'train_steps_per_second': 1.36, 'total_flos': 2215119840325632.0, 'train_loss': 0.03352261301211892, 'epoch': 3.0})

In [14]:
model.save_pretrained("./spam_model")
tokenizer.save_pretrained("./spam_model")

('./spam_model/tokenizer_config.json',
 './spam_model/special_tokens_map.json',
 './spam_model/vocab.txt',
 './spam_model/added_tokens.json',
 './spam_model/tokenizer.json')

In [17]:
!zip -r spam_model.zip spam_model

  adding: spam_model/ (stored 0%)
  adding: spam_model/special_tokens_map.json (deflated 42%)
  adding: spam_model/tokenizer_config.json (deflated 75%)
  adding: spam_model/vocab.txt (deflated 53%)
  adding: spam_model/config.json (deflated 45%)
  adding: spam_model/tokenizer.json (deflated 71%)
  adding: spam_model/model.safetensors (deflated 8%)


# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the model and tokenizer from the saved folder
model = AutoModelForSequenceClassification.from_pretrained("./spam_model")
tokenizer = AutoTokenizer.from_pretrained("./spam_model")

# Make sure the model is in evaluation mode (important for inference)
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [15]:
import torch

def predict_spam(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Run the model to get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()

    # Map the prediction to human-readable labels
    label_map = {0: "Not Spam", 1: "Spam"}

    return label_map[predicted_class_id]

In [16]:
predict_spam("Win a free vacation now by texting YES!")

'Spam'

In [17]:
predict_spam("Hey, want to grab coffee tomorrow?")

'Not Spam'

In [22]:
predict_spam("Is your butt too big? Our product can help with that")

'Not Spam'

In [23]:
predict_spam("You won a free giftcard!")

'Spam'

In [24]:
!zip -r spam_model.zip spam_model

  adding: spam_model/ (stored 0%)
  adding: spam_model/special_tokens_map.json (deflated 42%)
  adding: spam_model/tokenizer_config.json (deflated 75%)
  adding: spam_model/vocab.txt (deflated 53%)
  adding: spam_model/config.json (deflated 45%)
  adding: spam_model/tokenizer.json (deflated 71%)
  adding: spam_model/model.safetensors (deflated 8%)
